In [1]:
import cv2
import numpy as np
import urllib 

In [2]:
prototxt_path='models/colorization_deploy_v2.prototxt'
model_path='models/colorization_release_v2.caffemodel'
kernal_path='models/pts_in_hull.npy'
image_path='/home/darik/Desktop/Dev/B/W image colorizer/test4.jpg'


In [3]:
#using open cv documentation
net=cv2.dnn.readNetFromCaffe(prototxt_path,model_path)
points=np.load(kernal_path)
points=points.transpose().reshape(2,313,1,1)
net.getLayer(net.getLayerId('class8_ab')).blobs=[points.astype(np.float32)]
net.getLayer(net.getLayerId('conv8_313_rh')).blobs=[np.full([1,313],2.606,dtype="float32")]


In [4]:
bw_image=cv2.imread(image_path)
normalized=bw_image.astype("float32")/255.0
lab=cv2.cvtColor(normalized,cv2.COLOR_BGR2LAB)
resized_im=cv2.resize(lab,(224,224))
L=cv2.split(resized_im)[0]
L-=50
net.setInput(cv2.dnn.blobFromImage(L))
ab=net.forward()[0,:,:,:].transpose((1,2,0))
ab=cv2.resize(ab,(bw_image.shape[1],bw_image.shape[0]))
L=cv2.split(lab)[0]
colorized=np.concatenate((L[:,:,np.newaxis],ab),axis=2)
colorized=cv2.cvtColor(colorized,cv2.COLOR_LAB2BGR)
colorized=(255.0*colorized).astype("uint8")


In [5]:
cv2.imshow("BW Image",bw_image)
cv2.imshow("Colorized",colorized)
cv2.waitKey(0)
cv2.destroyAllWindows()
